$$
\renewcommand{\mat}[1]{\boldsymbol {#1}}
\renewcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\renewcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\renewcommand{\vec}[1]{\boldsymbol {#1}}
\renewcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\renewcommand{\rvar}[1]{\mathrm {#1}}
\renewcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\renewcommand{\diag}{\mathop{\mathrm {diag}}}
\renewcommand{\set}[1]{\mathbb {#1}}
\renewcommand{\cset}[1]{\mathcal{#1}}
\renewcommand{\norm}[1]{\left\lVert#1\right\rVert}
\renewcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\renewcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\renewcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\renewcommand{\given}[]{\,\middle\vert\,}
\renewcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\renewcommand{\grad}[]{\nabla}
\renewcommand{\norm}[1]{\left\lVert#1\right\rVert}
$$

# Part 2: Summary Questions
<a id=part2></a>

This section contains summary questions about various topics from the course material.

You can add your answers in new cells below the questions.

**Notes**

- Clearly mark where your answer begins, e.g. write "**Answer:**" in the beginning of your cell.
- Provide a full explanation, even if the question doesn't explicitly state so. We will reduce points for partial explanations!
- This notebook should be runnable from start to end without any errors.

### CNNs

1. Explain the meaning of the term "receptive field" in the context of CNNs.

**Answer:** The receptive field in context of CNNs refers to the portion/region of the previous layer in the NN that needs to be used in order to calculate the next feature. There is a closed formula for this: 
$$r_{l-1}=s_l \cdot r_l + (k_l - s_l)$$ 
Where $l$ is the index of the current layer, $r_l$ is the receptive field of the lth layer, $k_l$ is the kernel size, $s_l$ is the stride of layer l. 
Thus the equation can be further developed arithmetically by solving the recurrence equation and express the receptive field of the network, $r_0$ as follows:
$$r_0 = \sum_{l=1}^{L}\Bigl((k_l - 1) \prod_{i=1}^{l-1} s_i\Bigl) +1$$
The receptive field is used as a measure to understand how much of the initial input is used when computing an output. In general a larger receptive field indicates that more complex or global features are being used. For example, in image detection/classification it can indicate the complex features of the entire image are used and not just smaller/local patterns. 

TODO: double check calculations with regard to dilation

2. Explain and elaborate about three different ways to control the rate at which the receptive field grows from layer to layer. Compare them to each other in terms of how they combine input features.

**Answer:** Three ways to control the rate at which the receptive field grows are: Pooling, strides and dilation. Also it worth mentioning that trivially is we increase the kernel size the RF will also grow.

Pooling is adding adding a pooling layer to the networks architecture, this pooling layer consists of a pooling function which in general replaces the output of the network in that location with a summary statistic of nearby outputs. For example we can look at a max pooling function which returns the maximum output within a specified area (usually a rectangle) in the area. This also helps improve NNs because it can add invariance to small translations in the input. In terms of receptive field (RF) this can help improve the NN by increasing the RF because it considers the inputs neighbors when producing the output of the layer. Thus we can control the RF by increasing or decreasing the pooling functions we choose to use. In general as we increase the window of the pooling we increase the RF and conversely when decreasing.

Stride is the spatial distance that a convolution filter moves/steps in between consecutive applications of the filter the kernel. Informally it can be regarded as the number of cells that the filter moves vertically/horizontally between each convolution operation. A large stride will thus produce a smaller output of that convolution layer because the filter will complete the operations on the output with less applications. If we choose a small stride we will receive a larger RF and opposite if we use a large stride. 

Dilation is the spacing between kernel elements when applying to input. Informally it adds gaps/holes in the filter. This spaces out the area the convolution layer sees and thus increases the receptive field. However we can notice that since the kernel is simply spread out and not otherwise altered, we gain an increase in the RF without adding new parameters to the NN. By increasing the dilation we increase the RF and vice versa. This method can help capture large-scale features. 

When comparing the three methods and how they combine input features we can note that Pooling comes at the expense of adding an additional layer thus further adding parameters to our NN when increasing the receptive field. A small stride will increase the RF but also increase the parameters of the NN because the output layer  of the convolution will have more neurons. Dilation on the other hand has no effect on the number or parameters and when increased will increase the RF too. 

In terms of how they combine input features: Pooling usually takes some sort of norm of the pooling area and thus can loose spatial resolution of the network, but this helps the NN deal with small changes in the input and helps prevent over fitting. Stride applies the convolution function on a smaller number of elements thus can reduce computational cost and make the NN more efficient. Dilation will combine the input features in a larger-scale way and thus can create non conventional and global features and improve efficiency.

3. Imagine a CNN with three convolutional layers, defined as follows:

In [1]:
import torch
import torch.nn as nn

cnn = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, stride=2, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=7, dilation=2, padding=3),
    nn.ReLU(),
)

cnn(torch.rand(size=(1, 3, 1024, 1024), dtype=torch.float32)).shape

/Users/arielsernoff/miniconda3/lib/python3.9/site-packages/torch/nn/modules/conv.py:137: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:77.)
  self.weight = Parameter(torch.empty(


torch.Size([1, 32, 122, 122])

What is the size (spatial extent) of the receptive field of each "pixel" in the output tensor?

**Answer:**




4. You have trained a CNN, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$, and $f_l(\cdot;\vec{\theta}_l)$ is a convolutional layer (not including the activation function).

  After hearing that residual networks can be made much deeper, you decide to change each layer in your network you used the following residual mapping instead $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)+\vec{x}$, and re-train.

  However, to your surprise, by visualizing the learned filters $\vec{\theta}_l$ you observe that the original network and the residual network produce completely different filters. Explain the reason for this.

**Answer:** We can see that with the residual mapping we receive a new mapping that references the input given to that layer. This is the implementation of ResNet in general. The reason that the residual network produces completely different filters is because now that the network can reference the input which is like adding additional information or a use of "memory" when constructing features. Thus the residual network extracts features that also regard the residual mapping that was given to it, which can lead to very different filters. ResNets usually correct the vanishing gradient issue and allow to build a much deeper network which can also lead to the different features extracted and filters constructed. 

### Dropout

1. Consider the following neural network:

In [2]:
import torch.nn as nn

p1, p2 = 0.1, 0.2
nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Dropout(p=p1),
    nn.Dropout(p=p2),
)

Sequential(
  (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Dropout(p=0.1, inplace=False)
  (3): Dropout(p=0.2, inplace=False)
)

If we want to replace the two consecutive dropout layers with a single one defined as follows:
```python
nn.Dropout(p=q)
```
what would the value of `q` need to be? Write an expression for `q` in terms of `p1` and `p2`.

**Answer:** The parameter p passed to the dropout layer represents the probability that the element in the tensor (feature) will be zeroed. This has been proven to be an effective method to prevent overfitting by adding stochasticity to the model. Then if we regard the two events of dropout as independent (they are consecutive) we can infer from basic statistic rule that q need to equal the probability that feature is dropped in the first or second dropout layer. We can calculate this by calculating the probability of not being dropped in both layers and then calculating the chance of that not happening. Then:
$$1-q = (1-p_1) * (1-p_2) \Rightarrow q = 1 - (1-p_1) * (1-p_2) = 1 - 0.9 * 0.8 = 1-.72 = 0.28$$ 

2. **True or false**: dropout must be placed only after the activation function.

**Answer:** False. You can place dropout before or after an activation function. First with some activation functions there is no difference (for example, ReLU). In some cases, placing dropout layers before specific activation functions can improve the stability of the network if the input to the activation function has very large variance and the activation function amplifies this variance, then dropping out some values can reduce this effect. However it is always dependant on the specific case and in some instances it may be beneficial to place it before, for example if the activation function behaves in a detrimental way on values of 0.

3. After applying dropout with a drop-probability of $p$, the activations are scaled by $1/(1-p)$. Prove that this scaling is required in order to maintain the value of each activation unchanged in expectation.

**Answer:** The issue arises due to the fact that when using the dropout regularization method we only wish to drop certain nodes in the network during training and retain the full network during testing. When we apply the drop out we alter the expected value of the weight matrices of training and to compensate for this alteration we thus need to scale the activations. 

Proof: 
We will assume under contradiction that scaling does not need to be done and will show that the expected value of the new activations is not equal to the old expected value without drop out, thus showing that to hold accuracy during testing we need to scale.

Let $y_{i}$ be the vector of activations in the i'th node in a layer and $E[y_i]$ be the expected value of the activation and $E[y]$ is then the expected value of the entire vector of activations. 

After dropout is used on the layer the values are now defined:

let $d_i$ be a random variable deciding if the node will be activated or not. i.e. it is an indicator with probability of $1-p$. Note that $d_i$ is independent of $y_i$. Then, 

$$y_{i}^{'} =  \begin{cases}
      y_i, & \text{if}\ d_i=1 \\
      0, & \text{if}\ d_i=0
    \end{cases}$$

Then when calculating the expectation of the whole layer we get. 

$$E[y^{'}] = \sum_{i=0}^{n} E[y^{'}_{i}] = \sum_{i=0}^{n} E[y_i \cdot d_i] + E[0 \cdot (1-d_i)] \\
= \sum_{i=0}^{n} E[y_i \cdot (1-p)] + E[0 \cdot (p)] = (1-p)E[y]$$

Which is contradiction to our premise that we can use drop out without regularization without scaling because this leads to a different expected value of the activation vectors between training and testing. 

Thus to achieve equal expectation we must scale by a factor of $\frac{1}{(1-p)}$ leading to,

$$y_{i}^{'} =  \begin{cases}
      \frac{1}{(1-p)}\cdot y_i, & \text{if}\ d_i=1 \\
      0, & \text{if}\ d_i=0
    \end{cases}$$

$$E[y^{'}] = \sum_{i=0}^{n} E[y^{'}_{i}] = \sum_{i=0}^{n} E[\frac{1}{(1-p)}\cdot y_i \cdot d_i] + E[0 \cdot (1-d_i)] \\
= \sum_{i=0}^{n} E[\frac{1}{(1-p)}\cdot y_i \cdot (1-p)] + E[0 \cdot (p)] = E[y] $$

$\blacksquare$






### Losses and Activation functions

1. You're training a an image classifier that, given an image, needs to classify it as either a dog (output 0) or a hotdog (output 1). Would you train this model with an L2 loss? if so, why? if not, demonstrate with a numerical example. What would you use instead?

**Answer:** No, the benefits of an L2 loss are that it provides a weighted loss based on how "far" the output if from the true expected value. In classification we usually implement our models using a soft-max activation (or some  function returning a probability) in the last layer to extract the class that the network finds most probable to be the true label/class. In this case it doesn't matter how "close" the classifier was to the true label if it was mistaken, it classifies as the class with highest probability. L2 does not regard the fact that the output is a probability. In that sense we don't want to give weight to the classes and instead "punish" an incorrect classification equally regardless of how close it was. L2 loss is more suited for regression problems and problems with continuous outputs.  

As a numerical example we can look at these two prediction vectors and notice how using the L2 norm as a loss function yields a smaller loss on the second vector even though the classification would be wrong. 

Ground Truth = [1, 0, 0, 0]  

V1 = [0.3, 0.25, 0.25, 0.2] , L2 = 0.168  

V2 = [0.49, 0.5, 0.01, 0.0] , L2 = 0.128  

A more fitting loss function would be binary cross-entropy loss. Binary and also regular cross-entropy measure the difference between the predicted and true probability distributions, thus it takes in account the probabilistic nature of classification. 

Another reason why L2 loss is not optimal compared to binary cross-entropy loss is that L2 is sensitive to outliers where cross-entropy is less sensitive to them.

2. After months of research into the origins of climate change, you observe the following result:

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/de/PiratesVsTemp%28en%29.svg/1200px-PiratesVsTemp%28en%29.svg.png?20110518040647" /></center>

You decide to train a cutting-edge deep neural network regression model, that will predict the global temperature based on the population of pirates in `N` locations around the globe.
You define your model as follows:

In [3]:
import torch.nn as nn

N = 42  # number of known global pirate hot spots
H = 128
mlpirate = nn.Sequential(
    nn.Linear(in_features=N, out_features=H),
    nn.Sigmoid(),
    *[
        nn.Linear(in_features=H, out_features=H), nn.Sigmoid(),
    ]*24,
    nn.Linear(in_features=H, out_features=1),
)

While training your model you notice that the loss reaches a plateau after only a few iterations.
It seems that your model is no longer training.
What is the most likely cause?

**Answer:** It is most likely due to the fact that the model is very large and that we are using the sigmoid activation function. The sigmoid activation function saturates at large positive and negative values. The sigmoid function has non-steep gradients and suffers heavily from the vanishing gradient problem, especially in cases like this with deep and wide networks with many hidden layers. The biggest gradient of the sigmoid function is 0.25 at $x=0$ and we can see by looking at the functions derivative that it would quickly have a gradient that limits to 0. 

3. Referring to question 2 above: A friend suggests that if you replace the `sigmoid` activations with `tanh`, it will solve your problem. Is he correct? Explain why or why not.

**Answer** He is incorrect. The tanh function does in fact handle gradients better than the sigmoid, due to the greater derivatives of the function, but it also suffers from the vanishing gradient problem. The tanh activation function is often used in hidden layers so in this case because the majority of the model lies in one bulk of hidden layers it may work since it leaves the gradients between -1 and 1. However since the tanh function is also a type of sigmoidal function it often can act similarly to the sigmoid and most likely wont solve the issue. Usually using a ReLU or similar type of activation function is recommended. Also batch-normalization and proper weight initialization may help the network.

4. Regarding the ReLU activation, state whether the following sentences are **true or false** and explain:
  1. In a model using exclusively ReLU activations, there can be no vanishing gradients.
  1. The gradient of ReLU is linear with its input when the input is positive.
  1. ReLU can cause "dead" neurons, i.e. activations that remain at a constant value of zero.

**Answer:** 

5. False: ReLU activation functions can suffer from something similar to the vanishing gradient issue and it is often coined the dying ReLU problem. The ReLU function gives a gradient of 0 to all negative and zero values. Which makes those nodes no longer continue the update the weights and stop responding to the training. In deep networks this can lead to a difficulty in training the model properly and reaching good results.

6. False: First the ReLU is not differentiable at 0. Furthermore the gradient of the ReLU function is 0 for all negative values and 1 for all positive values. Thus we do not get a linear function due to the discrete case nature of the gradient and the original function. 

7. True: That is precisely the dying ReLU problem noted above. All neurons that get a negative weight value will no longer update due to the gradient of the ReLU function being 0 for negative values. This will stop the backpropogation updating of those neurons. This is often attempted to be overcome by using Leaky Relu, ELU or similar activation functions. Also this can be dealt with in part with proper weight initialization. 

### Optimization

1. Explain the difference between: stochastic gradient descent (SGD), mini-batch SGD and regular gradient descent (GD).

**Answer:** GD, SGD and mini-batch SGD (mbSGD) are all similar optimization algorithms used to optimize the weights of a network during training. The main difference between them is how many samples they regard while computing the next step. To elaborate, all of the methods take a number of samples, calculate the gradient of the samples and take a step in the steepest direction (greatest gradient). While GD calculates the gradients for all samples in the training set, SGD approximates the step direction based of of one sample, and mbSGD is a hybrid of both methods that takes a sub-sample of the training set and calculates the direction to step based of all the gradients in the subgroup. The pay off is like in most cases; efficiency over accuracy. Calculating the gradients for all samples can prove to be very computationally exhausting and can even lead to local minimas. The stochastic methods are much more efficient in most cases and can often lead out of local minimas. However, it has been proven that the local minima issue in most cases is not as large of an issue as often perceived and that the difference between the local and global minimas is often not very substantial.  

2. Regarding SGD and GD:
  1. Provide at least two reasons for why SGD is used more often in practice compared to GD.
  2. In what cases can GD not be used at all?

**Answer: 3**  
Reason 1: As stated above the computational efficiency of SGD often prevails over GD's accuracy in practice because GD calculates the gradients of all the training set while SGD only calculates the gradient of a single sample (or subset in mbSGD)
Reason 2: Also as stated above SGD has the ability to get out of local minimas because of the fact that it can overlook the steepest step of the training sample as a whole and "mistakenly" step out of the local minima. 

**Answer: 4**  
An instance where GD is not usable at all is when the data set/ training set is very large and thus computationally practically impossible to compute the gradients. In cases like this mbSGD offers a good solution to optimization.


3. You have trained a deep resnet to obtain SoTA results on ImageNet.
While training using mini-batch SGD with a batch size of $B$, you noticed that your model converged to a loss value of $l_0$ within $n$ iterations (batches across all epochs) on average.
Thanks to your amazing results, you secure funding for a new high-powered server with GPUs containing twice the amount of RAM.
You're now considering to increase the mini-batch size from $B$ to $2B$.
Would you expect the number of of iterations required to converge to $l_0$ to decrease or increase when using the new batch size? explain in detail.

**Answer:** In general we would expect the model to converge faster meaning the number or iterations would decrease. A larger mini-batch size allows to calculate more representative gradients in the model and thus step in a more accurate direction to the minima. However, it is worth noting that if 2B is too large it can be similar to the issues that arise when implementing regular GD (getting stuck in local minimas and computational time). It is also worth noting that twice the ram doesn't necessarily mean that the computations will be twice as fast (or the same speed for a batch twice as large) there can be different bottlenecks that can effect computational time. In addition, it is worth noting that other hyperparameters should be tweaked in accordance to the new mini-batch size, like learning rate for example.

4. For each of the following statements, state whether they're **true or false** and explain why.
  1. When training a neural network with SGD, every epoch we perform an optimization step for each sample in our dataset.
  1. Gradients obtained with SGD have less variance and lead to quicker convergence compared to GD.
  1. SGD is less likely to get stuck in local minima, compared to GD.
  1. Training  with SGD requires more memory than with GD.
  1. Assuming appropriate learning rates, SGD is guaranteed to converge to a local minimum, while GD is guaranteed to converge to the global minimum.
  1. Given a loss surface with a narrow ravine (high curvature in one direction): SGD with momentum will converge more quickly than Newton's method which doesn't have momentum.

**Answer:**

5. True: when using SGD the optimization step is calculated based of one single sample in the data set. mbSGD uses a specific batch size of samples that are shuffled between each iteration. This is specifically the advantage of SGD over GD we do not need to compute the step from all the data set. However an epoch if defined as an iteration through the whole training data set thus we will take a step for each sample.

6. False: Since SGD uses a subset of the data the gradient calculated is less accurate and thus doesn't necessarily step in the most optimal direction. Usually GD will converge in less iterations but it will also require more computational time/power. SGD will thus have more variance and will usually require more steps to converge to the minima but it may take less real time due to the easier computations.

7. True: Due the the fact that SGD uses a random subset (or single sample) to calculate the gradient it usually will not compute the steepest gradient and compute the next step direction that doesn't go directly to the minima, by so it can step out of/over a local minima. 

8. False: Due to the fact that GD uses the entire dataset to compute the gradient it needs to load the entire data set into the memory for the gradient calculation, whereas the SGD only loads a subset.

9. False: Neither are guaranteed to converge to a global minima, if GD gets caught in a local minima it will not reach the global one. Even SGD is not guaranteed to converge to a local minima because it is dependant on the subset of samples that computes the gradient in each step, which can introduce noise. Theoretically the model can oscillate around the global or a local minima or saddle point. 

10. True: Generally, In the case of a narrow ravine SGD with momentum can utilize the momentum built in the direction of the ravine to avoid oscillations in and out of the ravine. Newtons method which uses second order approximations is prone to being caught in the mentioned situation and take less optimal steps due to it being more sensitive to the curvature of the loss field. Overall the momentum can guide and allow the optimization algorithm to speed up in the direction of the ravine and avoid oscillations and saddle points. However neither of these methods are promised to prevail over the other.  

5. In tutorial 5 we saw an example of bi-level optimization in the context of deep learning, by embedding an optimization problem as a layer in the network.
  **True or false**: In order to train such a network, the inner optimization problem must be solved with a descent based method (such as SGD, LBFGS, etc).
  Provide a mathematical justification for your answer.

**Answer:** False, when solving a bi-level optimization problem we can regard the out optimization problem as the main problem with respect to the inner problem, then in effect we can use the chain rule to solve through the entire problem. To use the chain rule we must ensure that the optimization method of the inner problem is differentiable, SGD and LBFGS are both examples of differentiable optimization methods. Some different methods are not differentiable, like simulated annealing. However if it is worth noting that it can be solved without these methods if we can formulate a closed solution of the gradient, this is a vary rare and impractical solution though. Thus the answer is technically false but practically true.

6. You have trained a neural network, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$ for some arbitrary parametrized functions $f_l(\cdot;\vec{\theta}_l)$.
  Unfortunately while trying to break the record for the world's deepest network, you discover that you are unable to train your network with more than $L$ layers.
  1. Explain the concepts of "vanishing gradients", and "exploding gradients".
  2. How can each of these problems be caused by increased depth?
  3. Provide a numerical example demonstrating each.
  4. Assuming your problem is either of these, how can you tell which of them it is without looking at the gradient tensor(s)?

**Answer::** 

7. Vanishing and exploding gradients are when the gradients of the lower levels of a network become very small or very large respectfully. In the case of vanishing gradients the gradient become so small that the weights of the lower levels do not update during back propagation, in contrast the exploding gradient problem occurs when the gradient are so large that the weights update in a away that diverges and the model becomes unstable.

8. Since the vanishing/exploding gradient problems occur due to the gradient approaching 0 or infinity and being backpropagated through the layers, as we increase the depth of the model we increase the number of updates of the gradient which in return can amplify large or small gradients. If in a small network a "bad" gradient may only propagate through a few layers, in a large network it may have a massive impact on the entire networks optimization due to the exponential amplification that occurs and its great effect on early layers.

9. Numerical Examples:
-Vanishing Gradient:
If we take a deep network lets say with 10 layer, use the sigmoid activation function and use randomly initiated weights

-Exploding Gradient:

10. We can tell which problem it is based on the loss function during training. If we see that the loss function is stagnant we can assume we have a vanishing gradients because the model is no longer updating the weights matrices due to the gradients approaching 0. If the loss function is fluctuating but not diverging/improving we can assume that our issue is exploding gradients due to the model itself not diverging to a minima because the gradients that are being propagated are extremely large.

### Backpropagation

1. You wish to train the following 2-layer MLP for a binary classification task:
  $$
  \hat{y}^{(i)} =\mat{W}_2~ \varphi(\mat{W}_1 \vec{x}^{(i)}+ \vec{b}_1) + \vec{b}_2
  $$
  Your wish to minimize the in-sample loss function is defined as
  $$
  L_{\mathcal{S}} = \frac{1}{N}\sum_{i=1}^{N}\ell(y^{(i)},\hat{y}^{(i)}) + \frac{\lambda}{2}\left(\norm{\mat{W}_1}_{F}^{2} + \norm{\mat{W}_2}_{F}^{2} \right)
  $$
  Where the pointwise loss is binary cross-entropy:
  $$
  \ell(y, \hat{y}) =  - y \log(\hat{y}) - (1-y) \log(1-\hat{y})
  $$
  
  Write an analytic expression for the derivative of the final loss $L_{\mathcal{S}}$ w.r.t. each of the following tensors: $\mat{W}_1$, $\mat{W}_2$, $\mat{b}_1$, $\mat{b}_2$, $\mat{x}$.

**Answer:**
Denote the input to the activation function $\varphi$ as $z$ and denote $\varphi'$ as the derivative of the chosen activation function. 
Note: Since we are using a binary cross entropy loss function it is likely that $\varphi$ is the sigmoid function. 

For further use to develope expressions based of the chain rule let's develop the following expression:

$$\frac{\partial \ell(y^{(i)},\hat{y}^{(i)})}{\partial \hat{y}^{(i)}}=-\frac{y}{\hat{y}}+\frac{(1-y)}{(1-\hat{y})}\\
=\frac{-y(1-\hat{y})+\hat{y}(1-y)}{\hat{y}(1-\hat{y})} = \frac{-y+y\hat{y}+\hat{y}-\hat{y}y}{\hat{y}(1-\hat{y})}\\
=\frac{\hat{y}-y}{\hat{y}(1-\hat{y})}$$

Now using the chain rule we can develope the required partial derivatives:

- $\frac{\partial L_\mathcal{S}}{\partial W_{1}}$:
    - $\frac{\partial\hat{y}^{(i)}}{\partial \mat{W_1}} = \frac{\partial(\mat{W_2}\varphi(\mat{W_1}x^{i}+b_1)+b_2)}{\partial\mat{W_1}}=\mat{W_2}\varphi'(\mat{W_1}x^{i}+b_1)\cdot x $ 
    
    - $\frac{\partial L_\mathcal{S}}{\partial W_{1}} = \frac{\partial(\frac{1}{N}\sum_{i=1}^{N}\ell(y^{(i)},\hat{y}^{(i)})+\frac{\lambda}{2}\left(\norm{\mat{W}_1}_{F}^{2} + \norm{\mat{W}_2}_{F}^{2} \right))}{\partial W_1} = \frac{1}{N}\sum_{i=1}^{N}\frac{\partial(\ell(y^{(i)},\hat{y}^{(i)}))}{\partial\hat{y}^{(i)}}\cdot\frac{\partial\hat{y}^{(i)}}{\partial W_1} + \lambda W_1 = \frac{1}{N}\sum_{i=1}^{N}\frac{\hat{y}^{(i)}-y^{(i)}}{\hat{y}^{(i)}(1-\hat{y}^{(i)})}\mat{W_2}x^i\varphi'(\mat{W_1}x^{i}+b_1) + \lambda W_1$ 
  
    
        

- $\frac{\partial L}{\partial W_{2}}$:
    - $\frac{\partial\hat{y}^{(i)}}{\partial \mat{W_2}} = \frac{\partial(\mat{W_2}\varphi(\mat{W_1}x^{i}+b_1)+b_2)}{\partial\mat{W_2}}=\varphi'(\mat{W_1}x^{i}+b_1)$ 
    
    - $\frac{\partial L_\mathcal{S}}{\partial W_{2}} = \frac{\partial(\frac{1}{N}\sum_{i=1}^{N}\ell(y^{(i)},\hat{y}^{(i)})+\frac{\lambda}{2}\left(\norm{\mat{W}_1}_{F}^{2} + \norm{\mat{W}_2}_{F}^{2} \right))}{\partial W_2} = \frac{1}{N}\sum_{i=1}^{N}\frac{\partial(\ell(y^{(i)},\hat{y}^{(i)}))}{\partial\hat{y}^{(i)}}\cdot\frac{\partial\hat{y}^{(i)}}{\partial W_2} + \lambda W_2 = \frac{1}{N}\sum_{i=1}^{N}\frac{\hat{y}^{(i)}-y^{(i)}}{\hat{y}^{(i)}(1-\hat{y}^{(i)})}\varphi(\mat{W_1}x^{i}+b_1) + \lambda W_2$ 

- $\frac{\partial L}{\partial b_{1}}$:
    - $\frac{\partial z^{(i)}}{\partial b_1} = \frac{\partial\mat{W_1}x^{(i)}+b_1}{\partial\mat{b_1}}=1$

    - $\frac{\partial L_\mathcal{S}}{\partial b_{1}} = \frac{\partial(\frac{1}{N}\sum_{i=1}^{N}\ell(y^{(i)},\hat{y}^{(i)})+\frac{\lambda}{2}\left(\norm{\mat{W}_1}_{F}^{2} + \norm{\mat{W}_2}_{F}^{2} \right))}{\partial b_1} = \frac{1}{N}\sum_{i=1}^{N}\frac{\partial(\ell(y^{(i)},\hat{y}^{(i)}))}{\partial\hat{y}^{(i)}}\cdot\frac{\partial\hat{y}^{(i)}}{\partial z^{(i)}}\cdot \frac{\partial z^{(i)}}{\partial b_1} = \frac{1}{N}\sum_{i=1}^{N}\frac{\hat{y}^{(i)}-y^{(i)}}{\hat{y}^{(i)}(1-\hat{y}^{(i)})}\mat{W_2}\varphi'(z)$ 
    


- $\frac{\partial L}{\partial b_{2}}$:
    - $\frac{\partial\hat{y}^{(i)}}{\partial b_2} = \frac{\partial\mat{W_2}\varphi(\mat{W_1}x^{(i)}+b_1)+b_2}{\partial\mat{b_2}}=1$

    - $\frac{\partial L_\mathcal{S}}{\partial b_{2}} = \frac{\partial(\frac{1}{N}\sum_{i=1}^{N}\ell(y^{(i)},\hat{y}^{(i)})+\frac{\lambda}{2}\left(\norm{\mat{W}_1}_{F}^{2} + \norm{\mat{W}_2}_{F}^{2} \right))}{\partial b_2} = \frac{1}{N}\sum_{i=1}^{N}\frac{\partial(\ell(y^{(i)},\hat{y}^{(i)}))}{\partial\hat{y}^{(i)}}\cdot\frac{\partial\hat{y}^{(i)}}{\partial b_2} = \frac{1}{N}\sum_{i=1}^{N}\frac{\hat{y}^{(i)}-y^{(i)}}{\hat{y}^{(i)}(1-\hat{y}^{(i)})}$ 

- $\frac{\partial L}{\partial x}$:
    - $\frac{\partial\hat{y}^{(i)}}{\partial x^{(i)}} = \frac{\partial\mat{W_2}\varphi(\mat{W_1}x^{(i)}+b_1)+b_2}{\partial\mat{b_2}}=\mat{W_1}\cdot\varphi'(\mat{W_1}x^{(i)+b_1})$

    - $\frac{\partial L_\mathcal{S}}{\partial x^{(i)}} = \frac{\partial(\frac{1}{N}\sum_{i=1}^{N}\ell(y^{(i)},\hat{y}^{(i)})+\frac{\lambda}{2}\left(\norm{\mat{W}_1}_{F}^{2} + \norm{\mat{W}_2}_{F}^{2} \right))}{\partial x^{(i)}} = \frac{1}{N}\sum_{i=1}^{N}\frac{\partial(\ell(y^{(i)},\hat{y}^{(i)}))}{\partial\hat{y}^{(i)}}\cdot\frac{\partial\hat{y}^{(i)}}{\partial x^{(i)}} = \frac{1}{N}\sum_{i=1}^{N}\frac{\hat{y}^{(i)}-y^{(i)}}{\hat{y}^{(i)}(1-\hat{y}^{(i)})}\mat{W_1}\cdot\varphi'(\mat{W_1}x^{(i)}+b_1)$ 



2. The derivative of a function $f(\vec{x})$ at a point $\vec{x}_0$ is
  $$
  f'(\vec{x}_0)=\lim_{\Delta\vec{x}\to 0} \frac{f(\vec{x}_0+\Delta\vec{x})-f(\vec{x}_0)}{\Delta\vec{x}}
  $$
  
  1. Explain how this formula can be used in order to compute gradients of neural network parameters numerically, without automatic differentiation (AD).
  
  2. What are the drawbacks of this approach? List at least two drawbacks compared to AD.

**Answer:** 
1. Let $\vec{\theta}$ be a vector consisting of all the variables in the network. Specifically we can denote each individual variable in the network as $\theta _i$ and let $N$ be the number of variables in the network.

Then we can define the gradient of the loss function $L$ w.r.t. each parameter as follows: 
$$\nabla L_{\vec{\theta}}=(\frac{\partial{L}}{\partial{\theta _1}},...,\frac{\partial{L}}{\partial{\theta _N}})$$

Using the provided equation to calculate the partial derivative of each parameter to then obtain the gradient of the function. i.e.

$$\frac{\partial{L}}{\partial{\theta _i}} = \lim_{\Delta\vec{\theta}\to 0} \frac{L(\vec{\theta}+\Delta{\theta _i})-L(\vec{\theta})}{\Delta\vec{\theta}}$$

Then we can use the gradient to define the direction to step in based off of our optimization algorithm. 

2. The drawbacks of this approach are first that this is computationally taxing. In order to obtain this we need to compute the loss function for each parameter. 

In addition the method can be inaccurate. First, not all loss functions are differentiable at all points of their domain meaning that the method could give imprecise results. Also, this method requires to choose a value $\Delta$ as a parameter. It can be difficult to choose the right magnitude for this parameter, if it's too small the gradient may suffer from rounding errors and if it is too large we will loose accuracy of the gradient.



3. Given the following code snippet:
  1. Write a short snippet that implements that calculates gradient of `loss` w.r.t. `W` and `b` using the approach of numerical gradients from the previous question.
  2. Calculate the same derivatives with autograd.
  3. Show, by calling `torch.allclose()` that your numerical gradient is close to autograd's gradient.

In [12]:
import torch

N, d = 100, 5
dtype = torch.float64
X = torch.rand(N, d, dtype=dtype)
W, b = torch.rand(d, d, requires_grad=True, dtype=dtype), torch.rand(d, requires_grad=True, dtype=dtype)

def foo(W, b):
    return torch.mean(X @ W + b)

loss = foo(W, b)
print(f"{loss=}")

# TODO: Calculate gradients numerically for W and b
delta = 1e-6
grad_W = torch.zeros_like(W)
grad_b = torch.zeros_like(b)

# gradient w.r.t. W
for i in range(grad_W.shape[0]):
    for j in range(grad_W.shape[1]):
        W_temp = W.clone()
        W_temp[i,j] += delta
        loss_temp = foo(W_temp, b)
        grad_W[i,j] = (loss_temp-loss)/delta

# gradient w.r.t. b
for i in range(grad_b.shape[0]):
    b_temp = b.clone()
    b_temp[i] += delta
    loss_temp = foo(W, b_temp)
    grad_b[i] = (loss_temp-loss)/delta

# TODO: Compare with autograd using torch.allclose()
loss.backward()
autograd_W = W.grad
autograd_b = b.grad
assert torch.allclose(grad_W, autograd_W)
assert torch.allclose(grad_b, autograd_b)


loss=tensor(1.7437, dtype=torch.float64, grad_fn=<MeanBackward0>)


### Sequence models

1. Regarding word embeddings:
  1. Explain this term and why it's used in the context of a language model.
  1. Can a language model like the sentiment analysis example from the tutorials be trained without an embedding (i.e. trained directly on sequences of tokens)? If yes, what would be the consequence for the trained model? if no, why not?

**Answer:**
2. Embedding is the process of encoding tokens/words into tensors/multi-dimensional vectors in a way that preserves the some of the word's semantic attributes. Generally, each dimension is attributed to some feature of the token. This is used in the context of a language model because using this method allows the model to utilize the semantic values and the context of the words to preform it's required tasks in a more precise way. Without word embedding the tokens are do not hold their relations to one another. 
3. Yes is can be trained without using embedding however it would probably not reach a desired result. To train the model without embedding we would likely use one-hot encoding like we did in the homework assignments. This would lead to two major drawbacks. 

First, the model would not be able to recognize and utilize the semantic attributes of the words. This would lead to a less effective model. We can remember from the assignments that when we built a text generator of a Shakespeare play the model made specific patterns similar to the play appear but with no logic behind the wording, similarly the model for semantic analysis would not find the logic in the reviews and would struggle greatly with new words or new sequences of recognized words. 

Secondly, the model would be inefficient. The input layer of a one-hot encoded model is the size of all the words in the batch multiplied by the size of the vocabulary (because of the one-hot encoding). In contrast, a model using word embedding has an input layer with the size of the dimension of the embedding vector multiplied by the number of words in the batch. This is usually substantially smaller since generally the vocabulary is much larger than the number of features we attribute to the embedding. However it is worth noting that this is not always this case and depends on the specific task the model is trying to solve. 

2. Considering the following snippet, explain:
  1. What does `Y` contain? why this output shape?
  2. How you would implement `nn.Embedding` yourself using only torch tensors. 

In [13]:
import torch.nn as nn

X = torch.randint(low=0, high=42, size=(5, 6, 7, 8))
embedding = nn.Embedding(num_embeddings=42, embedding_dim=42000)
Y = embedding(X)
print(f"{Y.shape=}")

Y.shape=torch.Size([5, 6, 7, 8, 42000])


**Answer:** 
2.1. Y contains a tensor that holds all of the input values from X but in a dense vector/embedded tensor. There are 42 words in the vocabulary here so embedding holds a table with the corresponding embedding for each word in the vocabulary. Applying embedding on the input vector X (which holds the sequences and batches) returns a corresponding tensor of the input X but with the multi-dimensional (42000) embedding of each token. 

2.2. It is possible to do it with a Matrix of size (num of embeddings, embedding dim) or (42,42000) like in the above example. And then when we want to get the value of a specific token we need to look up the input vector X in it using indexing (passing the input tensor as the index to the tensor we want to get the embedding from). Usually a common way to initialize the embedding matrix can be to use random values to ensure there is no predetermined relationships between words or use a pre-trained embedding matrix. 


3. Regarding truncated backpropagation through time (TBPTT) with a sequence length of S: State whether the following sentences are **true or false**, and explain.
  1. TBPTT uses a modified version of the backpropagation algorithm.
  2. To implement TBPTT we only need to limit the length of the sequence provided to the model to length S.
  3. TBPTT allows the model to learn relations between input that are at most S timesteps apart.

**Answer:**
3.1 True, the concept is similar to using mini-batch SGD. We calculate the gradient and backpropagate back through the model only by a defined number of layers. This helps minimize computational time and help deal with the vanishing/exploding gradient problems that often occur in sequence models due to them back propagating through time and thus the gradient for the loss can be dependant on many layers. 

3.2 False, we need to use the whole sequence and can even use an infinitely long sequence. The implementation is significant in that we update the gradient after a fixed length of layers. But we need the whole sequence to reach optimal results. 

3.3 False, the gradient is only calculated for up to S timestaps apart and thus each iteration only learns new relations that are at most S apart. However over multiple iterations, the model in whole can learn relations that are global. It does however hinder the models ability to provide accurate answers theoretically but in a practical setting it allows the model to reach better results since the input data may be so large that otherwise training is impossible and even when using truncation it picks up on local and global relations. 

### Attention

1. In tutorial 7 (part 2) we learned how to use attention to perform alignment between a source and target sequence in machine translation.
  1. Explain qualitatively what the addition of the attention mechanism between the encoder and decoder does to the hidden states that the encoder and decoder each learn to generate (for their language). How are these hidden states different from the model without attention?
  
  2. After learning that self-attention is gaining popularity thanks to the shiny new transformer models, you decide to change the model from the tutorial: instead of the queries being equal to the decoder hidden states, you use self-attention, so that the keys, queries and values are all equal to the encoder's hidden states (with learned projections). What influence do you expect this will have on the learned hidden states?


**Answer:**

1.1 The addition of attention allows the model to generate different and potentially more precise hidden states. The mechanism allows the decoder to use more precise/relevant information gained from the encoder to better align the text. The decoder can regard a specific portion of the encoder hidden states instead of having to regard the entire sequence summarized into a single state by the encoder. This is a very intuitive approach as it is similar to how we would translate information; instead of first processing the information in its whole we would divide it into relevant parts and translate in a segmented manner.

1.2 Using the encoders hidden states as the keys, queries and values of the model allows the model to attend all parts of the encoders output sequence as opposed to using queries from the decoder hidden states where the models attention is determined by the current decoder state. We can assume that this change will allow the learned hidden states to capture more distant relationships between tokens. Often in the decoder models use a both types of blocks to achieve better results.

### Unsupervised learning

1. As we have seen, a variational autoencoder's loss is comprised of a reconstruction term and  a KL-divergence term. While training your VAE, you accidentally forgot to include the KL-divergence term.
What would be the qualitative effect of this on:

  1. Images reconstructed by the model during training ($x\to z \to x'$)?
  1. Images generated by the model ($z \to x'$)?

**Answer:**

2. During training we would get better results. The model would work like a regular auto-encoder since the KL-divergence term is the factor that ensures the distribution that is being learned in the VAE is similar to a specified distribution. The reconstructed images in training would be more similar to the input images because the model would have "overfit" the learned dist. to the input data's dist. 

3. The generated images however would suffer significantly from this. The reconstructed images would appear less diverse, clear and similar to what we would expect. The latent space generated by the model would not follow a specific distribution and thus loose the ability to create images with diversity and expected features. The model learned a latent space in accordance to the input models and loose from the stochastic benefits of the VAE. 



2. Regarding VAEs, state whether each of the following statements is **true or false**, and explain:
  1. The latent-space distribution generated by the model for a specific input image is $\mathcal{N}(\vec{0},\vec{I})$.
  2. If we feed the same image to the encoder multiple times, then decode each result, we'll get the same reconstruction.
  3. Since the real VAE loss term is intractable, what we actually minimize instead is it's upper bound, in the hope that the bound is tight.

**Answer:**

3. False, the loss function in a VAE consists of a reconstruction loss and KL-divergence. The model does not simply learn the normal or otherwise specified distribution as the latent-space but also aims to learn a distribution that also minimizes reconstruction error but is close to or "follows" the distribution given. Otherwise we would lose the meaningful information of the input that we aim to learn.

4. False, the model has the stochastic element of sampling which is what allows it to generate different representations and is the underlying concept in the VAE. The model may then generate a different latent representation even for the same image, it is worth noting though that they will most likely hold similarities since the model still aims to minimize the reconstruction loss.

5. True, the true loss is intractable because the KL-divergence which measures the difference between the learned and given distribution is often intractable due to the computation of the integrals needed to be calculated in the expression. Thus the upper bound of the loss and specifically the KL-divergence can be regarded as an optimization problem in the model that is attempted to be limited. 

2. Regarding GANs, state whether each of the following statements is **true or false**, and explain:
  1. Ideally, we want the generator's loss to be low, and the discriminator's loss to be high so that it's fooled well by the generator.
  2. It's crucial to backpropagate into the generator when training the discriminator.
  3. To generate a new image, we can sample a latent-space vector from $\mathcal{N}(\vec{0},\vec{I})$.
  4. It can be beneficial for training the generator if the discriminator is trained for a few epochs first, so that it's output isn't arbitrary.
  5. If the generator is generating plausible images and the discriminator reaches a stable state where it has 50% accuracy (for both image types), training the generator more will further improve the generated images.

**Answer:**

3. False. When training a GAN we are essentially solving a min max problem: 
$$\underset{G}{min} \; \underset{D}{max} \; V(D,G)$$ 
Where,
$$ V(D,G) = \mathbb{E}_{x∼p(x)}[logD(x)]+\mathbb{E}_{z∼q(z)}[log(1−D(G(z)))]$$
Thus we do want the generators loss to be low and by this achieving a generator which is able to fool the discriminator and have a high loss to the discriminator indicating that the discriminator is indeed being fooled. However, a high loss of the discriminator is not necessarily good. We want a good discriminator model that preforms well in discerning between the synthesized and input images. Thus we do not want any model with a high discriminator loss. 

4. False. When training the discriminator (D) we do not want to update the generator's weights thus do not want to backprogate through it. When training the D we are in some sort providing feedback to the generator (G) and thus if we were to backprop through it and update its weights with regard to the D's training we would be "overfitting" the model and loose the adversarial component of the network. 

5. True. In most GAN implementations the network takes an input of a latent vector and generates an image from it.

6. True. Usually the discriminator is initialized randomly and thus in the first epochs of training it does not provide any beneficial feedback to the generator. Thus by training for a few epochs prior to training the G is often beneficial to ensure that the training process more efficient and that the D is able to discern between synthesized and real input before training the G.

7. False. Once reaching a steady state of 50% the generator has "beaten" the discriminator, for the discriminator could be randomly guessing real/fake and reach 50 percent. Thus further training will not improve the loss/accuracy but, it still may improve the images quality or features/details it generates. This is not always the case though so the answer is False. 

